# GoogLeNet

In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchsummary import summary

import renyan_utils as ry

## Inception Module

In [2]:
class Inception(nn.Module):
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # line 1
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size = 1)
        # line 2
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size = 1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size = 3, padding = 1)
        # line 3
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size = 1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size = 5, padding = 2)
        # line 4
        self.p4_1 = nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size = 1)
        
    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim = 1)

## GoogLeNet

- MaxPool2d(kernel_size = 3, stride = 2) links between modules (5 in total)
- Inception Modules assure that the size of images will correspond with each other. We only have to pay attention to channel numbers

In [3]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1))

In [4]:
# line 2 in Inception
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size = 1),
                 nn.Conv2d(64, 192, kernel_size = 3, padding = 1),
                 nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1))

In [5]:
# connect 2 inception modules in series
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                  Inception(256, 128, (128, 192), (32, 96), 64),
                  nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1))

In [6]:
# connect 5 inception modules in series
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                  Inception(512, 160, (112, 224), (24, 64), 64),
                  Inception(512, 128, (128, 256), (24, 64), 64),
                  Inception(512, 112, (144, 288), (32, 64), 64),
                  Inception(528, 256, (160, 320), (32, 128), 128),
                  nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1))

In [7]:
# connect 2 inception modules in series
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                  Inception(832, 384, (192, 384), (48, 128), 128),
                  ry.GlobalAvgPool2d())

In [8]:
net = nn.Sequential(b1, b2, b3, b4, b5, 
                    ry.FlattenLayer(),
                   nn.Linear(1024, 10))

In [12]:
summary(net, (1, 96, 96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           3,200
              ReLU-2           [-1, 64, 48, 48]               0
         MaxPool2d-3           [-1, 64, 24, 24]               0
            Conv2d-4           [-1, 64, 24, 24]           4,160
            Conv2d-5          [-1, 192, 24, 24]         110,784
         MaxPool2d-6          [-1, 192, 12, 12]               0
            Conv2d-7           [-1, 64, 12, 12]          12,352
            Conv2d-8           [-1, 96, 12, 12]          18,528
            Conv2d-9          [-1, 128, 12, 12]         110,720
           Conv2d-10           [-1, 16, 12, 12]           3,088
           Conv2d-11           [-1, 32, 12, 12]          12,832
        MaxPool2d-12          [-1, 192, 12, 12]               0
           Conv2d-13           [-1, 32, 12, 12]           6,176
        Inception-14          [-1, 256,

In [15]:
batch_size = 16
train_iter, test_iter = ry.load_data_fashion_mnist_resize(batch_size, resize = 96)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
ry.train_mnist_net(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)